In [1]:
#%%
#!pip3 install -r requirements.txt
#import dropbox
from selenium.webdriver.common.keys import Keys
# auxiliary functions modified by Luis.
import scrape_functions as kzd
import sys
import calendar
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import re
from selenium.webdriver.support.ui import Select
import random
import time
from selenium import webdriver
import os
import numpy as np
import importlib
importlib.reload(sys.modules['scrape_functions'])
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime

In [2]:
def BookingReport(deal_boxes):
    page_report = []
    for deal_box in deal_boxes:
        hotel_name = deal_box.find_element(By.CSS_SELECTOR,
            'div[data-testid="title"]'
        ).get_attribute('innerHTML').strip()

        if not deal_box.find_elements(By.CSS_SELECTOR, 'span[data-testid="price-and-discounted-price"]'):
            hotel_price = deal_box.find_element(By.CSS_SELECTOR,
                'div[data-testid="price-and-discounted-price"]'
            ).find_element(By.TAG_NAME, 'div').get_attribute('innerHTML').strip()
        else:
            hotel_price = deal_box.find_element(By.CSS_SELECTOR,
                'span[data-testid="price-and-discounted-price"]'
            ).get_attribute('innerHTML').strip()
            hotel_price = hotel_price.replace("€&nbsp;", "")
            hotel_price = hotel_price.replace(",", "")
            hotel_price = int(hotel_price)

        try:
            hotel_score = deal_box.find_element(By.CSS_SELECTOR,
                'div[aria-label*="Scored"]'
            ).get_attribute('innerHTML').strip()
        except:
            hotel_score = 'nan'
        page_report.append(
            [hotel_name, hotel_price, hotel_score]
        )
        # print([hotel_name, hotel_price, hotel_score])
    return page_report

def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    time.sleep(1)
    return(total_pages)

def click(browser,element_type, element):
    thing = browser.find_element(element_type, element)
    browser.execute_script("arguments[0].click()", thing)

def zoom_out(browser):
    browser.execute_script("document.body.style.zoom='30%'")

In [3]:
#%%
# %% Starting code
cities = ['Paris', 'Barcelona']

i=0
for place in cities:
    if place == 'Barcelona':
        i=0
    check_in_date = ["2023-05-15", "2023-05-29"]
    check_out_date = ["2023-05-21", "2023-06-04"]
    
    for d in check_in_date:
        link = 'https://www.booking.com/'
        browser = webdriver.Chrome()
        browser.get(link)

        zoom_out(browser)
        time.sleep(2)
        destination_input = browser.find_element('xpath','//input[@placeholder="Where are you going?"]')
        destination_input.send_keys(place)


        click(browser, 'xpath', '//button[@type="submit"]')

        #time.sleep(2)

        zoom_out(browser)

        week=[check_in_date[i], check_out_date[i]]
        
        today_month = datetime.now().month
        while today_month < int(week[0].split('-')[1]):
            time.sleep(2)
            dates = browser.find_elements('xpath','//table[@class="aadb8ed6d3"]/tbody/tr/td/span')
            print('dates: ', len(dates))

            click(browser, 'xpath', '//button[@class="fc63351294 a822bdf511 e3c025e003 fa565176a8 cfb238afa1 ae1678b153 c9fa5fc96d be298b15fa"]')
            print('month clicked')
            today_month+=1

        #%%

        click(browser, 'xpath', '//span[@data-date="{}"]'.format(week[0]))
        click(browser, 'xpath', '//span[@data-date="{}"]'.format(week[1]))

        time.sleep(1)

        # click on empty space
        click(browser, 'xpath', '//input[@class="ce45093752"]')
        print('blank space clicked')

        time.sleep(1)

        search_button = browser.find_element('xpath', '//button[@type="submit"]')
        browser.execute_script("arguments[0].click()", search_button)
        print('search button clicked')

        time.sleep(2)

        pages = get_number_pages(browser)
        print('total pages: ', pages)
        #%%

        #%%        
        # Loop pages
        full_report = []

        for page in range(pages):
            # Get the list of hotel elements on the page

            print('Scraping page {} out of {}...'.format(page+1, pages))
            print('before: ', len(full_report))

            browser.implicitly_wait(2)

            zoom_out(browser)

            deal_boxes = browser.find_elements(
                    By.XPATH,
                    "//div[@data-testid='property-card']"
                    )
            report = BookingReport(deal_boxes)
            time.sleep(4)

            full_report.extend(report)

            print('after: ', len(full_report))

            click(browser, 'xpath', '//button[@aria-label="Next page"]')

            time.sleep(4)

        report_df = pd.DataFrame(full_report)

        # %%
        deal_boxes = browser.find_elements(
                    By.XPATH,
                    "//div[@data-testid='property-card']")

        report = BookingReport(deal_boxes)
        report_df = pd.DataFrame(full_report, columns=['hotel','price','rating'])
        report_df['start_date'] = week[0]
        report_df['city']=place
        df = report_df.copy()
        j=0
        for r in df['price']:
            if type(r) is int:
                pass
            else:
                df['price'][j] = df['price'][j].replace("Price €&nbsp;", "")
                df['price'][j] = df['price'][j].replace(",", "")
                df['price'][j] = int(df['price'][j])
            j+=1
        df.to_csv('scrape{}v2.csv'.format(place+week[0]))
        i+=1
df1 = pd.read_csv('scrapeParis2023-05-15v2.csv')
df2 = pd.read_csv('scrapeParis2023-05-29v2.csv')
df3 = pd.read_csv('scrapeBarcelona2023-05-15v2.csv')
df4 = pd.read_csv('scrapeBarcelona2023-05-29v2.csv')
df1 = df1.append(df2)
df1 = df1.append(df3)
df1 = df1.append(df4)
df1.to_csv('scrape_all_cities.csv')

dates:  59
month clicked
dates:  61
month clicked
dates:  61
month clicked
blank space clicked
search button clicked
total pages:  40
Scraping page 1 out of 40...
before:  0
after:  25
Scraping page 2 out of 40...
before:  25
after:  50
Scraping page 3 out of 40...
before:  50
after:  75
Scraping page 4 out of 40...
before:  75
after:  100
Scraping page 5 out of 40...
before:  100
after:  125
Scraping page 6 out of 40...
before:  125
after:  150
Scraping page 7 out of 40...
before:  150
after:  175
Scraping page 8 out of 40...
before:  175
after:  200
Scraping page 9 out of 40...
before:  200
after:  225
Scraping page 10 out of 40...
before:  225
after:  250
Scraping page 11 out of 40...
before:  250
after:  275
Scraping page 12 out of 40...
before:  275
after:  300
Scraping page 13 out of 40...
before:  300
after:  325
Scraping page 14 out of 40...
before:  325
after:  350
Scraping page 15 out of 40...
before:  350
after:  375
Scraping page 16 out of 40...
before:  375
after:  400
Scr

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@data-date="2023-05-21"]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x010A6643]
	(No symbol) [0x0103BE21]
	(No symbol) [0x00F3DA9D]
	(No symbol) [0x00F71342]
	(No symbol) [0x00F7147B]
	(No symbol) [0x00FA8DC2]
	(No symbol) [0x00F8FDC4]
	(No symbol) [0x00FA6B09]
	(No symbol) [0x00F8FB76]
	(No symbol) [0x00F649C1]
	(No symbol) [0x00F65E5D]
	GetHandleVerifier [0x0131A142+2497106]
	GetHandleVerifier [0x013485D3+2686691]
	GetHandleVerifier [0x0134BB9C+2700460]
	GetHandleVerifier [0x01153B10+635936]
	(No symbol) [0x01044A1F]
	(No symbol) [0x0104A418]
	(No symbol) [0x0104A505]
	(No symbol) [0x0105508B]
	BaseThreadInitThunk [0x759D00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77447BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77447B8E+238]


In [4]:
from datetime import datetime
type()

int